### RAG (Retrieval Augmented Generation)
## In This
- Part A: We will divide our documents into CHUNKS
- Part B: We will encode our CHUNKS into VECTORS and put in Chroma
- Part C: We will visualize our vectors

### PART A: Divide our documents into chunks

In [1]:
import os
import glob
import tiktoken
import numpy as np
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [2]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-oss:120b-cloud"
db_name = "vector_db"
load_dotenv(override=True)
OLLAMA_API_KEY = os.getenv('OLLAMA_API_KEY')
if not OLLAMA_API_KEY:
    raise Exception('Missing api key')

In [3]:
# How many characters in all the documents?

knowledge_base_path = "knowledge-base/**/*.md"
files = glob.glob(knowledge_base_path, recursive=True)
print(f"Found {len(files)} files in the knowledge base")

entire_knowledge_base = ""

for file_path in files:
    with open(file_path, 'r', encoding='utf-8') as f:
        entire_knowledge_base += f.read()
        entire_knowledge_base += "\n\n"

print(f"Total characters in knowledge base: {len(entire_knowledge_base):,}")

Found 76 files in the knowledge base
Total characters in knowledge base: 304,434


In [4]:
# How many tokens in all the documents?

encoding = tiktoken.encoding_for_model("gpt-oss-120b")
tokens = encoding.encode(entire_knowledge_base)
token_count = len(tokens)
print(f"Total tokens for {MODEL}: {token_count:,}")

Total tokens for gpt-oss:120b-cloud: 63,555


In [5]:
# Load in everything in the knowledgebase using LangChain's loaders

folders = glob.glob("knowledge-base/*")

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

print(f"Loaded {len(documents)} documents")

Loaded 76 documents


In [6]:
documents[1]

Document(metadata={'source': 'knowledge-base\\company\\careers.md', 'doc_type': 'company'}, page_content="# Careers at Insurellm\n\n## Why Join Insurellm?\n\nAt Insurellm, we're not just building software—we're revolutionizing an entire industry. Since our founding in 2015, we've evolved from a high-growth startup to a lean, profitable company with 32 highly talented employees managing 32 active contracts across all eight of our product lines.\n\nAfter reaching 200 employees in 2020, we strategically restructured in 2022-2023 to focus on sustainable growth, operational excellence, and building a world-class remote-first culture. Today, we're a tight-knit team of exceptional professionals who deliver outsized impact through automation, AI, and strategic focus on high-value enterprise clients—from regional insurers to global reinsurance partners.\n\n### Our Culture\n\nWe live by our core values every day:\n- **Innovation First**: We encourage experimentation and creative problem-solving\

In [7]:
# Divide into chunks using the RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"Divided into {len(chunks)} chunks")
print(f"First chunk:\n\n{chunks[0]}")

Divided into 413 chunks
First chunk:

page_content='# About Insurellm

Insurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. Its first product was Markellm, the marketplace connecting consumers with insurance providers.

The company experienced rapid growth in its first five years, expanding its product portfolio to include Carllm (auto insurance portal), Homellm (home insurance portal), and Rellm (enterprise reinsurance platform). By 2020, Insurellm had reached a peak of 200 employees with 12 offices across the US.' metadata={'source': 'knowledge-base\\company\\about.md', 'doc_type': 'company'}


### PART B: Make vectors and store in Chroma
In Week 3, you set up a Hugging Face account and got an HF_TOKEN

In [17]:
# Pick an embedding model

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
#embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 413 documents


In [9]:
# Let's investigate the vectors

collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

There are 413 vectors with 384 dimensions in the vector store


### Part C: Visualize!

In [10]:
# Prework

result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']
doc_types = [metadata['doc_type'] for metadata in metadatas]
colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]

In [13]:
# We humans find it easier to visalize things in 2D!
# Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochastic neighbor embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)


fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': [orange, orange, orange, ..., blue, blue, blue],
                         'opacity': 0.8,
                         'size': 5},
              'mode': 'markers',
              'text': [Type: company<br>Text: # About Insurellm  Insurellm was
                       founded by Avery Lancaster in 2015 as an insurance tech
                       startup des..., Type: company<br>Text: However, the company
                       underwent a strategic restructuring in 2022-2023 to focus on
                       profitability and ..., Type: company<br>Text: - **Bizllm**
                       quickly gained traction with 7 commercial insurance
                       contracts, including regional carri..., ..., Type:
                       products<br>Text: ### Regulatory Compliance Tools Rellm
                       includes built-in compliance tracking features to help
                       organiz..., Type: products<br>Text: Join the growing number
                       of organizations leveraging Rellm to enhance their
                       reinsurance processes whi..., Type: products<br>Text:
                       Experience the future of reinsurance with Rellm, where
                       innovation meets reliability. Let Insurellm h...],
              'type': 'scatter',
              'x': {'bdata': ('n59BQb9WOkEar1dBXoBowHPassC5+N' ... 'dBmA07QVIug0EzhItBcBhNQb2cgkE='),
                    'dtype': 'f4'},
              'y': {'bdata': ('3tEqwPEoyb+F36W/EsouQdHLTEGwz6' ... 'nAPUMdwdjx2sCZ8InAxhMTwZkQUcA='),
                    'dtype': 'f4'}}],
    'layout': {'height': 600,
               'margin': {'b': 10, 'l': 10, 'r': 20, 't': 40},
               'scene': {'xaxis': {'title': {'text': 'x'}}, 'yaxis': {'title': {'text': 'y'}}},
               'template': '...',
               'title': {'text': '2D Chroma Vector Store Visualization'},
               'width': 800}
})

In [15]:
# Let's try 3D!

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=10, b=10, l=10, t=40)
)

fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': [orange, orange, orange, ..., blue, blue, blue],
                         'opacity': 0.8,
                         'size': 5},
              'mode': 'markers',
              'text': [Type: company<br>Text: # About Insurellm  Insurellm was
                       founded by Avery Lancaster in 2015 as an insurance tech
                       startup des..., Type: company<br>Text: However, the company
                       underwent a strategic restructuring in 2022-2023 to focus on
                       profitability and ..., Type: company<br>Text: - **Bizllm**
                       quickly gained traction with 7 commercial insurance
                       contracts, including regional carri..., ..., Type:
                       products<br>Text: ### Regulatory Compliance Tools Rellm
                       includes built-in compliance tracking features to help
                       organiz..., Type: products<br>Text: Join the growing number
                       of organizations leveraging Rellm to enhance their
                       reinsurance processes whi..., Type: products<br>Text:
                       Experience the future of reinsurance with Rellm, where
                       innovation meets reliability. Let Insurellm h...],
              'type': 'scatter3d',
              'x': {'bdata': ('idaLQS1EuUGZZL5BYlNgv8d+xMDmSP' ... 'dBi/yHQVvJLEEX0GJBitJqQWsXdUE='),
                    'dtype': 'f4'},
              'y': {'bdata': ('Y/OtwczYusHP5oPB+6WHwdauxME83W' ... 'zBjhNvwVulucDYsgDBfAkawQ9DWcE='),
                    'dtype': 'f4'},
              'z': {'bdata': ('Q9R1wQ8ZVcF7vGjBfpWtwbdJksHKhM' ... 'a++8eNwJuuCsE3a4PBKPDCwO15scE='),
                    'dtype': 'f4'}}],
    'layout': {'height': 700,
               'margin': {'b': 10, 'l': 10, 'r': 10, 't': 40},
               'scene': {'xaxis': {'title': {'text': 'x'}},
                         'yaxis': {'title': {'text': 'y'}},
                         'zaxis': {'title': {'text': 'z'}}},
               'template': '...',
               'title': {'text': '3D Chroma Vector Store Visualization'},
               'width': 900}
})